In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/social-network-ads/Social_Network_Ads.csv


In [2]:
import pandas as pd #data preprocessing
import numpy as np #linear algebra
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv('../input/social-network-ads/Social_Network_Ads.csv')

In [4]:
df.sample(5)

,Age,EstimatedSalary,Purchased
272,60,42000,1
55,24,55000,0
321,53,82000,1
137,30,107000,1
6,27,84000,0


*Here we can see the Age and EstimatedSalary feature are not standardized*

**0.1. Train Test Split** 


In [5]:
from sklearn.model_selection import train_test_split
X_train,  X_test, y_train, y_test =train_test_split(df.drop(['Purchased'],axis=1),df['Purchased'],test_size =0.3,random_state=0)
X_train.shape, y_test.shape

((280, 2), (120,))

**1. StandardScaler**

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
#fit the scaler to the trainset , it will learn the parameters
scaler.fit(X_train)

# transoforming the train and test sets

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [7]:
scaler.mean_

array([3.78642857e+01, 6.98071429e+04])